# Extração da característica relação sinal-ruído (SNR) de dados de EEG

O objetivo é utilizar um conjunto de dados com nível de atividade cerebral considerado basal e "subtraí-lo" do sinal obtido no experimento a fim de reconhecer os padrões de foco com maior clareza.

No contexto da caracterização de foco, podemos, a partir do sinal basal, classificar os sinais dos ritmos cerebrais com a presença ou não de foco, de forma que as amostras de sinais extraídas de um buffer sejam rotuladas com com a presença ou não de foco.

Esta análise será realizada com o auxílio do classificador SVM (_Support Vector Machine_). Nesse sentido, uma porcetagem das amostras são utilizadas para treino e outra para o teste (30 e 70% respectivamente).

In [ ]:
import mne
import numpy as np
from scipy.signal import welch
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [ ]:
# Transform a string that represents a time value ("min:sec") into number and converts it to seconds
def convert_min_to_sec(time):
    minutes, seconds = map(int, time.split(":"))
    return minutes * 60 + seconds

# Transfor a string thats represents a range time "0:21 - 0:40" into index that will be used to cut data
def convert_time_range_to_index(timerange):
    start, end = map(str, timerange.split(" - "))
    new_start = convert_min_to_sec(start)
    new_end = convert_min_to_sec(end)
    
    index = []
    index.append(new_start * 250)
    index.append(new_end * 250)

    return index

In [ ]:
n_channels = 8
ch_types = ['eeg'] * n_channels
sfreq = 250
ch_names = ["F3", "Fz", "F4", "C3", "Cz", "C4", "P3", "P4"]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
info.set_montage("standard_1020")

In [ ]:
ia = '../dataset-s7/IA/OpenBCI-RAW-2023-09-28_16-51-25_IA.txt'
ia_ob = np.loadtxt(ia, delimiter=',', skiprows=5, usecols=range(1, 9))

basal = '../dataset-s7/TF/OpenBCI-RAW-2023-11-07_13-17-01_TF.txt'
basal_ob = np.loadtxt(basal, delimiter=',', skiprows=5, usecols=range(1, 9))

In [ ]:
# Aula IA
ia_timeranges = [
    "6:00 - 10:17",
]

# Basal (TF)
basal_timerange = [
    "3:28 - 4:28"
]

In [ ]:
ia_index = []
ia_index = convert_time_range_to_index(ia_timeranges[1])

basal_index = []
basal_index = convert_time_range_to_index(basal_timerange[0])

data_cut_ia = ia_ob[ia_index[0]:ia_index[1], :]
data_cut_basal = basal_ob[basal_index[0]:basal_index[1], :]

X = {
    'ia': mne.io.RawArray(data_cut_ia.T, info),
    'basal': mne.io.RawArray(data_cut_basal.T, info),
}

### Estimando o ruído de fundo através do sinal basal

In [ ]:
# Lista que armazena as médias de potência para cada canal
noise_power = []

for channel_data in X['basal']: # precisa ser transposta? parece que ja ta transposta
    fft_result = np.fft.fft(channel_data)

    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    
    # média da potência no intervalo de tempo sem estímulo
    base_power = np.mean(psd)
    noise_power.append(base_power)

# média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
print(estimated_background_noise)

Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [ ]:
# agora vamos adaptar ambas características 
# aplicando para o nosso sinal de interesse

# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
target_amplitudes_adjusted = data_cut_ia - estimated_background_noise # data_cut_ia ou X['ia'] ?

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR)
print(wide_band_SNR.shape)

### Tarefa para aplicação das características SNR:

Agora que temos os dois vetores de características SNR, podemos utilizar buffers com e sem a evocação dos rítmos que caracterizam o foco.

#### Divisão dos dados

Utilizando a iteração (por exemplo, de 5 segundos caracterizada pela janela) realizada no sinal a cada ~1 segundo, realize a rotulação dos dados de interesse (Beta e Gamma). Ou seja, cada amostra sera um sinal de 5 segundos (1250 pontos de 8 canais). A janela que não for qualificada como Beta ou Gama por exemplo, poderá ser rotulada com "desfoque". Se acharem interessante, adicionar rótulos do ritmo Theta também.

No caso do sinal que representa o basal (se tiverem) poderá pegar um único sinal de aproximadamente 30 segundos para ser utilizado na equação de ruído, que irá ter como resultado um único valor. Lembrando que o valor de ruído deve atuar no sinal no domínio da frequência.

#### Classificação

Em nossos dados simulados, temos 150.000 pontos com 8 canais. A utilização desses dados funcionará da seguinte forma para a criação do vetor de características:

- 150.000 (pontos totais) / 250 (taxa de amostragem) = 600 segundos
- 600 / 5 (tamanho da janela sem sobreposição) = 120 amostras

| 1   | SNRw1                | SNRw2 | ... | SNRw8 | SNRn1 | SNRn2 | ... | SNRn8 |
|-----|----------------------|-------|-----|-------|-------|-------|-----|-------|
| 2   | [w1, w2, ..., w1250] |       |     |       |       |       |     |       |
| 3   |                      |       |     |       |       |       |     |       |
| ... |                      |       |     |       |       |       |     |       |
| 120 |                      |       |     |       |       |       |     |       |

- Agora transforme cada um dos vetores de pontos no domínio da frequência (1250 pontos) em um único valor real. Neste caso pode ser utilizado tanto a média como a mediana (ou ambos). Se utilizarmos as duas, teremos no final 32 colunas de características:
    - 8 canais
    - SNR narrow e SNR wide (2)
    - Média e mediana (2)

| 1   | 1   | ... | 32 |
|-----|-----|-----|----|
| 2   | w'  | ... |    |
| 3   | ... |     |    |
| ... |     |     |    |
| 120 |     |     |    |



Após obter o vetor de característica, realizar a divisão dos dados em treinamento e teste (normalmente uma proporção de 70 e 30% respectivamente) e aplicar para o classificador SVM.

**PLUS**: Ao final da tarefa, verificar a melhora dos resultados utilizando um seletor de características. Neste caso, podemos utilizar o RFE (*Recursive Feature Elimination*) em uma fase anterior a classificação para reduzir as 32 características se for necessário.

----

### Dúvidas

1 - `for channel_data in X['basal']: # precisa ser transposta? parece que ja ta transposta`
2 - `target_amplitudes_adjusted = data_cut_ia - estimated_background_noise # data_cut_ia ou X['ia'] ?`